# Overview

These notes are a continuation of the previous set of notes. We will go through some more advanced usage of SQLAlchemy, including mass insertion, and advanced query features. We will then look under the hood for some details on how SQLAlchemy's features are implemented, using interesting programming and embedding techniques.

# Music Library Example

Our running example will be a database made out of Artists and Albums. We define the schema below.

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import validates, relationship

import math

engine = create_engine('sqlite:///:memory:', echo=True)
Base = declarative_base()

class Artist(Base):
    __tablename__ = 'artists' # the actual name in the DB

    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    year = Column(Integer, nullable=True)

    @validates('year')
    def year_is_valid(self, key, year):
        year = math.floor(year)
        if year < 1950 or year > 2019:
            raise ValueError('Illegal year!')
        return year
    
    def __repr__(self): # python side
        return "<Artist(id={}, name='{}', year={}>".format(self.id, self.name, self.year)

class Album(Base):
    __tablename__ = 'albums'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    tracks = Column(Integer, nullable=False)

    artist_id = Column(Integer, ForeignKey(Artist.id), nullable=False)
    artist = relationship('Artist', backref='albums', lazy='joined') # backref is still lazy

    def __repr__(self):
        return "<Album(id={}, name={}, #tracks={}, artist={})".format(
            self.id, self.name, self.tracks, self.artist.name)

Base.metadata.create_all(engine)

2019-11-15 00:34:18,337 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-15 00:34:18,337 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,338 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-15 00:34:18,338 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,339 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("albums")
2019-11-15 00:34:18,339 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,340 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("albums")
2019-11-15 00:34:18,340 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,341 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("artists")
2019-11-15 00:34:18,341 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,341 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("artists")
2019-11-15 00:34:18,342 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,

## Inserting Data

Now, we will insert data into our database. A common technique is to read data from a csv or JSON file and insert it into DB. Sometimes, or to scrape data online or acquire it from other databases. Here, we will start with some JSON dump of the data.

In [2]:
json_str = '''[
    {"type": "artist", "content": {"name": "Lynyrd Skynyrd", "year": 1964}},
    {"type": "artist", "content": {"name": "Led Zeppelin", "year": 1968}},
    {"type": "artist", "content": {"name": "Metallica", "year": 1981}},
    {"type": "artist", "content": {"name": "Slayer", "year": 1981}},
    {"type": "artist", "content": {"name": "Pantera", "year": 1981}},
    {"type": "artist", "content": {"name": "Pearl Jam"}},
    {"type": "artist", "content": {"name": "Tool", "year": 1990}},
    {"type": "album", "content": {"name": "Master of Puppets", "artist": "Metallica", "tracks": 8}},
    {"type": "album", "content": {"name": "...And Justice for All", "artist": "Metallica", "tracks": 9}},
    {"type": "album", "content": {"name": "Fear Inoculumn", "artist": "Tool", "tracks": 10}},
    {"type": "album", "content": {"name": "Seasons in the Abyss", "artist": "Slayer", "tracks": 10}},
    {"type": "album", "content": {"name": "Hell Awaits", "artist": "Slayer", "tracks": 7}},
    {"type": "album", "content": {"name": "Far Beyond Driven", "artist": "Pantera"}},
    {"type": "album", "content": {"name": "Ten", "artist": "Pearl Jam"}}
]'''

Our JSON data contains many artists and bands. However, our data is not clean. Some artists are missing their year of formation, some albums may be missing the number of tracks. If an artist fails to be inserted, we want all its albums not to be inserted. However, if an album fails, this should not affect other albums or the artist.

We can achieve this via committing every entry independently, starting with bands first. More complicated conditions can be implemented using subtransactions.

In [3]:
import json

from sqlalchemy.orm import sessionmaker
SessionMaker = sessionmaker(bind=engine)

session = SessionMaker()

data = json.loads(json_str)
artists = {}
for entry in data:
    if entry['type'] == 'artist':
        artist = Artist(**entry['content'])
        artists[entry['content']['name']] = artist
        session.add(artist)
        try:
            session.commit()
        except Exception as e:
            print('\n', e, '\n')
            
            del artists[entry['content']['name']]
            session.rollback()

for entry in data:
    if entry['type'] == 'album':
        artist = entry['content']['artist']
        del entry['content']['artist']
        
        album = Album(**entry['content'])
        if artist in artists:
            album.artist = artists[artist]
            session.add(album)
            try:
                session.commit()
            except Exception as e:
                print('\n', e, '\n')
                session.rollback

session = SessionMaker()
print('\n', session.query(Artist).all(), '\n')
print(session.query(Album).all())

2019-11-15 00:34:18,566 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,569 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-15 00:34:18,570 INFO sqlalchemy.engine.base.Engine ('Lynyrd Skynyrd', 1964)
2019-11-15 00:34:18,574 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-15 00:34:18,577 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,579 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-15 00:34:18,581 INFO sqlalchemy.engine.base.Engine ('Led Zeppelin', 1968)
2019-11-15 00:34:18,586 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-15 00:34:18,589 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,591 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-15 00:34:18,592 INFO sqlalchemy.engine.base.Engine ('Metallica', 1981)
2019-11-15 00:34:18,596 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-15 

## Advanced Queries

We will make several queries that attempt to cover a variety of useful SQL features, including group by, aggregation, ordering, and limits.

### All Artists

We begin with something simple. We want to get all the artists in the database.

In [4]:
session = SessionMaker()

artists = session.query(Artist).all()
print('')
for artist in artists:
    print(artist)

2019-11-15 00:34:18,673 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,674 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists
2019-11-15 00:34:18,674 INFO sqlalchemy.engine.base.Engine ()

<Artist(id=1, name='Lynyrd Skynyrd', year=1964>
<Artist(id=2, name='Led Zeppelin', year=1968>
<Artist(id=3, name='Metallica', year=1981>
<Artist(id=4, name='Slayer', year=1981>
<Artist(id=5, name='Pantera', year=1981>
<Artist(id=6, name='Pearl Jam', year=None>
<Artist(id=7, name='Tool', year=1990>


### All Artists with Albums

We begin with something simple. We want to choose all artists with some albums.

In regular SQL, this requires us to perform a join on the foreign key between the artist and albums tables.
SQLAlchemy allows us to express the condition using the logical relation, without going through details of foreign key.

We can use .any() on the relationship, to get a condition that is satisfied if there relationship is not empty for a row.

In [5]:
from sqlalchemy import func

session = SessionMaker()

artists = session.query(Artist).filter(Artist.albums.any()).all()
for artist in artists:
    # .albums will cause an SQL query to be issued because it is lazy by default
    print('\n', artist, artist.albums, '\n')

2019-11-15 00:34:18,750 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,751 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE EXISTS (SELECT 1 
FROM albums 
WHERE artists.id = albums.artist_id)
2019-11-15 00:34:18,751 INFO sqlalchemy.engine.base.Engine ()
2019-11-15 00:34:18,753 INFO sqlalchemy.engine.base.Engine SELECT albums.id AS albums_id, albums.name AS albums_name, albums.tracks AS albums_tracks, albums.artist_id AS albums_artist_id 
FROM albums 
WHERE ? = albums.artist_id
2019-11-15 00:34:18,754 INFO sqlalchemy.engine.base.Engine (3,)

 <Artist(id=3, name='Metallica', year=1981> [<Album(id=1, name=Master of Puppets, #tracks=8, artist=Metallica), <Album(id=2, name=...And Justice for All, #tracks=9, artist=Metallica)] 

2019-11-15 00:34:18,755 INFO sqlalchemy.engine.base.Engine SELECT albums.id AS albums_id, albums.name AS albums_name, albums.tracks AS albums_t

### Artists With At Least Two Albums

This query is a bit different from the previous one, because it involves selecting the artist information, as well as information about the albums. SQLAlchemy provides us with a variety of aggregate functions, that can aggregate information at the database side, count is one such function, that returns the count of entries in a specific column.

Note that we are not interested in the total number of albums in the entire database, rather the total number of albums **per** artist. Therefore, we should group by artists and aggregate these groupings.

Lastely, the condition of having two or more albums can only be applied **after** the group by and aggregate is executed. SQL provides a *having* construct, which executes a filter after aggregation. SQLAlchemy provides having as well. SQLAlchemy provides a filter function as well, which is translated to an *where*, which gets executed prior to aggregation.

Finally, our query now referes to columns within the albums table. So we must instruct SQLAchemy to join with that table. SQLAlchemy automatically figures out how to perform this join, based on the logical relationships defined in the schema.

In [6]:
from sqlalchemy import func, text

session = SessionMaker()

# This is not recommended, because it uses SQL as a string
print('query 1')
query = session.query(Artist, func.count(Album.id).label('count')) # specify what to select
query = query.join(Album) # join with albums
query = query.group_by(Artist.id) # group based on artists
query = query.having(text('count > 1')) # keep only records with more than 1 album
artists = query.all()
for (artist, count) in artists:
    print(artist, count)

# A bit better
print('')
print('query 2')
count = func.count(Album.id) # count now can be utilized in python
query = session.query(Artist, count) #same as before
query = query.join(Album)
query = query.group_by(Artist.id)
query = query.having(count > 1) # same condition, but written in python
for (artist, count) in query.all():
    print(artist, count)

query 1
2019-11-15 00:34:18,835 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,838 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, count(albums.id) AS count 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id 
HAVING count > 1
2019-11-15 00:34:18,839 INFO sqlalchemy.engine.base.Engine ()
<Artist(id=3, name='Metallica', year=1981> 2
<Artist(id=4, name='Slayer', year=1981> 2

query 2
2019-11-15 00:34:18,852 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, count(albums.id) AS count_1 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id 
HAVING count(albums.id) > ?
2019-11-15 00:34:18,854 INFO sqlalchemy.engine.base.Engine (1,)
<Artist(id=3, name='Metallica', year=1981> 2
<Artist(id=4, name='Slayer', year=1981> 2


### Get The Two Bands With The Most Number of Tracks

The first part of this query is similar to the previous one, we want to join with the albums table, because it is the table with the tracks information. We also want to aggregate the albums grouped by their artists.

The difference now is that we want to sum the number of tracks, instead of using count. Additionally, we should order the produced rows by the sum, and pick only the top two.

In [7]:
session = SessionMaker()

tracks_sum = func.sum(Album.tracks)
query = session.query(Artist, tracks_sum).join(Album)
query = query.group_by(Artist.id)
query = query.order_by(tracks_sum.desc()) # order in descending order
query = query.limit(2) # get only the first 2 results
for (artist, tracks) in query.all():
    print(artist, tracks)

2019-11-15 00:34:18,922 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-15 00:34:18,923 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, sum(albums.tracks) AS sum_1 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id ORDER BY sum(albums.tracks) DESC
 LIMIT ? OFFSET ?
2019-11-15 00:34:18,924 INFO sqlalchemy.engine.base.Engine (2, 0)
<Artist(id=3, name='Metallica', year=1981> 17
<Artist(id=4, name='Slayer', year=1981> 17


# Behind The Scenes

Here we will look more at how SQLAlchemy implements certain features behind the scenes.

## Declarative Base

How does SQLAlchemy keep track of which python classes are modules? Modules inherit from a declarative base. However, inheritance is a passive operation. It does not notify the parent class of the act of inheritance (not at definition time at least).

The easy solution for this would be to require developers to manually register their modules, by calling some function. However, python provide a better automated solution via MetaClasses: [source](https://github.com/sqlalchemy/sqlalchemy/tree/master/lib/sqlalchemy/ext).

## Meta Classes

A Meta class, is a class whose instances are classes themselves. It is a class that gives classes instead of objects.

A meta class can be written in python by inheriting from type.

In [8]:
class RegisteringMetaClass(type):
    def __init__(cls, clsname, bases, attrs):
        print('register', clsname, bases, attrs.keys())
        return super().__init__(clsname, bases, attrs)

class MyClass(object, metaclass=RegisteringMetaClass):
    attr = 'my attribute'
    def func():
        pass

class MyClass2(MyClass):
    attr2 = 'other attribute'

class MyClass3():
    pass

register MyClass (<class 'object'>,) dict_keys(['func', 'attr', '__module__', '__qualname__'])
register MyClass2 (<class '__main__.MyClass'>,) dict_keys(['attr2', '__module__', '__qualname__'])


The above example uses the metaclass=<metaclass> syntax, which is explicit but not regular. SQLAlchemy classes inherit from Base directly without refering to MetaClass. This can be achieved by ensuring that Base is a class defined with metaclass=<Metaclass>, or by getting an instance of the metaclass (whose instances are classes themselves), and using it as the parent class in inheritance.

In [9]:
def mysetattr(self, key, value):
    print('instrumented', key, value)
    super(self.__class__, self).__setattr__(key, value)

class InstrumentingMetaClass(type):
    def __new__(cls, clsname, bases, attrs):
        if clsname != 'Base':
            print('instrument', clsname, '\n')
            attrs['__setattr__'] = mysetattr
        return super().__new__(cls, clsname, bases, attrs)

# instances of the meta class are indeed classes
MetaClassInstance = InstrumentingMetaClass('Base', (object,), {})
print('MetaClassInstance =', MetaClassInstance)

# inheriting from an instance of the metaclass applies the metaclass implicitly
# this is the same behavior as MyClass2 above
class ConcreteClass(MetaClassInstance):
    def __init__(self):
        self.my_attr = 'Test'

instance = ConcreteClass()
instance.my_attr = 'Test 2'

MetaClassInstance = <class '__main__.Base'>
instrument ConcreteClass 

instrumented my_attr Test
instrumented my_attr Test 2


## Query Implementation

Another interesting thing to look at is how queries are implemented. In particular, how they get built up incrementally and translated to SQL, and how column-based conditions and manipulations work.

### Query Class and Objects

Source code: [Query](https://github.com/zzzeek/sqlalchemy/blob/master/lib/sqlalchemy/orm/query.py).

The first thing to notice is that calling session.query() returns an object from sqlalchemy's Query class.

In [10]:
query = session.query(Artist)
print('type(query) =', type(query))

type(query) = <class 'sqlalchemy.orm.query.Query'>


Second, note that the Query class overrides \_\_str\_\_(), so that it dumps the SQL statment that is equivalent to the query object.

In [11]:
sql = str(query)
print(sql)

SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists


As expected, all the functionality we have been using before for querying is implemented as methods in the Query class. This includes filter, order_by, limit, join, all, and many others.

In [12]:
from sqlalchemy.orm.query import Query
print(Query.filter)
print(Query.order_by)
print(Query.limit)
print(Query.join)
print(Query.all)

<function Query.filter at 0x7f240bc17ea0>
<function Query.order_by at 0x7f240bc1a158>
<function Query.limit at 0x7f240bc19730>
<function Query.join at 0x7f240bc1a7b8>
<function Query.all at 0x7f240bc19b70>


### Columns and Conditions

In the queries we have seen so far, we regularly used typical python operators, such as > or ==, to define conditions on columns. This may seem weird at first. Semantically, we are used to using > and == to compare Python values eagerly. However, here they are being used in a lazy way, and their actual execution happens at the database side.

Let us try to analyze how this is implemented systematically.

First, note two important observations:
1. **&lt;ModuleClass&gt;.&lt;column_name&gt;** is an instance object from SQLAlchemy's instrumented attribute class. This allows SQLAlechmy to customize how this column is used. This is a bit surprising. Looking at the definition of Artist class above, we may be led to believe that this should be an instance of Column, since thats how we define this property. However, SQLAlchemy uses the fact that Base controls the metaclass of our modules, to instrument their properties at definition time. The instrumented property contains the column definition and its information.
2. **&lt;module_instance&gt;.&lt;column_name&gt;** is a python value corresponding to the value of that column in the respective row in the table.

In [13]:
artist = session.query(Artist).get(3)
print('')
print('type(artist.name), artist.name =', type(artist.name), artist.name)
print('type(Artist.name), Artist.name =', type(Artist.name), Artist.name)

2019-11-15 00:34:19,477 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.id = ?
2019-11-15 00:34:19,479 INFO sqlalchemy.engine.base.Engine (3,)

type(artist.name), artist.name = <class 'str'> Metallica
type(Artist.name), Artist.name = <class 'sqlalchemy.orm.attributes.InstrumentedAttribute'> Artist.name


This can lead to confusion at times, at the surface of it, it seems like .name should be the same in both instances and their class. However, python supports having static and object-bound attributes with the same name. Below is a small demonstration of that.

In [14]:
class MyClass():
    attr1 = 'Static attribute 1'
    attr2 = 'Static attribute 2'
    def __init__(self):
        self.attr1 = 'Object-bound attribute'

obj = MyClass()
print(MyClass.attr1, ',', MyClass.attr2)
print(obj.attr1, ',', obj.attr2)

        

Static attribute 1 , Static attribute 2
Object-bound attribute , Static attribute 2


Looking deeper at the content of the instrumented attributes, we can see that InstrumentedAttribute overrides a variety of operators, including >, <, and ==. This means that when we use these operators on instrumented attribute objects, SQLAlchemy functions are executed, and they define the semantics of those operators.

In [15]:
print(dir(Artist.name), '\n')
print(Artist.name.__le__) # this function overrides (Artist.name < things)

['__add__', '__and__', '__class__', '__clause_element__', '__contains__', '__delattr__', '__delete__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__radd__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__rshift__', '__rsub__', '__rtruediv__', '__selectable__', '__set__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '_adapt_to_entity', '_bulk_update_tuples', '_impl_uses_objects', '_is_internal_proxy', '_of_type', '_parententity', '_parentmapper', '_query_clause_element', '_supports_population', 'adapt_to_entity', 'adapter', 'all_', 'any', 'any_', 'any_op', 'asc', 'between', 'bool_op', 'class_', 'collate', 'comparator', 'concat', 'c

Finally, now we can see the effect of such an operator. It returns an SQLAlchemy object that keeps track of what operators were used, which when passed to the query object, are analyzed and added to the corresponding SQL statement.

This is very similar to our Lazy deep embedding from several weeks ago, and it is a common technique for runtime embedding of translators or analyzers.

In [16]:
#from sqlalchemy import and_

condition1 = Artist.year > 1980
print(type(condition1))
print(str(condition1))

query2 = query.filter(condition1)
print('')
print(type(query2))
print(str(query2))

condition2 = Artist.year <= 1990

<class 'sqlalchemy.sql.elements.BinaryExpression'>
artists.year > :year_1

<class 'sqlalchemy.orm.query.Query'>
SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.year > ?


The power of such a deep embedding is that its constructs can be composed and combined arbitrarily, in unison with other python constructs. For example, conditions may be passed between functions as parameters, or get created inside for loops or if statements.

Here is a simple example demonstrating this ability.

In [17]:
def create_condition(column):
    return column <= 1990

condition2 = create_condition(Artist.year)
print(str(condition2))

from sqlalchemy import and_
print('')
print(and_)

condition = and_(condition1, condition2)
print('')
print(type(condition))
print(str(condition))

artists.year <= :year_1

<function and_ at 0x7f24202c7598>

<class 'sqlalchemy.sql.elements.BooleanClauseList'>
artists.year > :year_1 AND artists.year <= :year_2


Finally, we can use these conditions when desired by passing them to queries. A condition is a plain Lazy object that does not posses any execution specific values, and therefore it can be re-used by several queries.

In [18]:
query = query.filter(condition)
print(type(query))
print(str(query))

print('')
result = query.all()

print('')
print(result)

<class 'sqlalchemy.orm.query.Query'>
SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.year > ? AND artists.year <= ?

2019-11-15 00:34:19,997 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.year > ? AND artists.year <= ?
2019-11-15 00:34:19,999 INFO sqlalchemy.engine.base.Engine (1980, 1990)

[<Artist(id=3, name='Metallica', year=1981>, <Artist(id=4, name='Slayer', year=1981>, <Artist(id=5, name='Pantera', year=1981>, <Artist(id=7, name='Tool', year=1990>]


### Other Interesting Implementation Details

There are many other interesting aspects of SQLAlchemy that are worth further discussion. For brievity, we only highlight a couple of them.

1. Lazy loading of relationships: properties (both static and object-bound) of module classes and objects are instrumented. This means that accessing them triggers a python hook defined within SQLAlchemy, which can modify how the access work. In particular, for object-bound attributes matching static attributes of relationships (e.g. artist.albums), this hook checks whether the relationship was previously loaded, and loads it from the database using SQL queries if it needs to, before returning the result. This can be achieved via reflection (e.g. by overriding \_\_getattr\_\_).

2. Migrations and schema consistency: SQLAlchemy builds a graph-like data structure that stores the defined modules and their dependencies. This is done via the metaclass. This gives SQLAlchemy the chance to check whether the schema has been modified after the database was created, since it can also query the database for schematic information, and compare the two. This gives SQLAlchemy the ability to generate and manage SQL statements for altering the schema. Because of how critical and dangerous schema editing can be, SQLAlchemy generate python files called "migrations", that can be run via SQLAlchemy manaully, to generate and execute SQL statments for altering the schema, within safe transaction blocks. You can read more about this [here](https://alembic.sqlalchemy.org/en/latest/tutorial.html).